# Práctica: Contrastes


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Cargar datos reales: Longley
longley = sm.datasets.longley.load_pandas().data.copy()
longley.head()


### Variables del dataset Longley
- `TOTEMP`: Empleo total (variable dependiente).  
- `GNPDEFL`: Deflactor del GNP.  
- `GNP`: GNP (Producto Nacional Bruto).  
- `UNEMP`: Desempleo (en miles).  
- `ARMED`: Fuerzas Armadas (en miles).  
- `POP`: Población.  
- `YEAR`: Año.


In [ ]:
# Comprobar nombres exactos de columnas en su instalación
longley.columns


## 1) Ajuste del modelo OLS 


In [ ]:
y = longley["TOTEMP"].astype(float)

X_cols = [c for c in longley.columns if c != "TOTEMP"]
X = longley[X_cols].astype(float)


# Ajustar OLS
model = sm.OLS(y, sm.add_constant(X))
results = model.fit()

results.summary()


## 2) Significación **global**

La prueba F en el resumen contrasta:  
$$
H_0: \beta_1=\beta_2=\cdots=\beta_k=0
$$
frente a la alternativa de que **algún** coeficiente es no nulo.


In [ ]:
print("Estadístico F global:", results.fvalue)
print("p-valor F global:", results.f_pvalue)

## 3) Significación **individual**

Para cada \( \beta_j \):  
$$
H_0: \beta_j = 0
$$ 


In [ ]:
coef_table = pd.DataFrame({
    "coef": results.params,
    "std_err": results.bse,
    "t": results.tvalues,
    "p>|t|": results.pvalues
})
coef_table


## 4) Contrastes sobre **relaciones lineales** \(R\beta = r\)

Además de probar si un coeficiente es 0, a menudo queremos saber si **dos efectos son iguales** o si **una combinación** de efectos cumple cierta relación.


In [ ]:
# Ver el orden de parámetros (índices de beta)
results.params.index


### 4.1) Igualdad de dos coeficientes: $\beta_{\text{GNP}} = \beta_{\text{UNEMP}}$

Interpretación: “¿Un incremento marginal en GNP tiene el **mismo** efecto sobre el empleo que un incremento marginal en UNEMP (desempleo)?” 


In [ ]:

names = results.params.index.tolist()

# Construir restricción: beta_GNP - beta_UNEMP = 0
R = np.zeros((1, len(names)))
R[0, names.index("GNP")]   = 1.0
R[0, names.index("UNEMP")] = -1.0
r = np.array([0.0])

ftest_eq = results.f_test((R, r))   # F-test de igualdad

print("== Igualdad de coeficientes: beta_GNP = beta_UNEMP ==")
print("F-stat:", ftest_eq.fvalue, "p-value:", ftest_eq.pvalue)


### 4.2) Grupo de coeficientes nulos: $\beta_{\text{GNP}} = \beta_{\text{UNEMP}} = 0$


In [ ]:
R = np.zeros((2, len(names)))
r = np.zeros(2)

R[0, names.index("GNP")]   = 1.0           # beta_GNP = 0
R[1, names.index("UNEMP")] = 1.0           # beta_UNEMP = 0

ftest_group_zero = results.f_test((R, r))
print("== Grupo cero: beta_GNP = beta_UNEMP = 0 ==")
print("F-stat:", float(ftest_group_zero.fvalue), "p-value:", float(ftest_group_zero.pvalue))


### 4.3) Combinación lineal: $\beta_{\text{GNPDEFL}} + \beta_{\text{POP}} = \beta_{\text{ARMED}}$



In [ ]:

# Restricción: beta_GNPDEFL + beta_POP - beta_ARMED = 0
R = np.zeros((1, len(names)))
R[0, names.index("GNPDEFL")] = 1.0
R[0, names.index("POP")]  = 1.0
R[0, names.index("ARMED")]  = -1.0
r = np.array([0.0])

ftest_combo = results.f_test((R, r))
print("== Combinación: beta_GNPDEFL + beta_POP = beta_ARMED ==")
print("F-stat:", float(ftest_combo.fvalue), "p-value:", float(ftest_combo.pvalue))
